Rotten tomatoes guide for top movies. 

In [41]:
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time

In [ ]:
#driver = webdriver.Chrome(ChromeDriverManager().install())
driver = webdriver.Chrome('/Applications/chromedriver')

TODO: paginate.

In [72]:
driver.get('https://www.rottentomatoes.com/browse/movies_at_home/critics:certified_fresh?page=5')

In [ ]:
## click button paginate manually for the movie links.
from selenium.common.exceptions import WebDriverException

for i in range(1, 2):  
    time.sleep(7)
    try:
        pag = driver.find_element(By.XPATH,'//div[@class="discovery__actions"]//button')
        driver.execute_script("arguments[0].click();", pag)
    except WebDriverException:
        break;       

movie_we = driver.find_elements(By.XPATH, '//a[@class = "js-tile-link"]')   
 
links = []
for i in range(len(movie_we)):
    links.append(movie_we[i].get_attribute('href'))
links    

#pd.DataFrame(links).to_csv('links.csv') # save the links to a csv file

In [64]:
movie_we = driver.find_elements(By.XPATH, '//a[@class = "js-tile-link"]') 

links = []
for i in range(len(movie_we)):
    links.append(movie_we[i].get_attribute('href'))

pd.DataFrame(links).to_csv('links.csv') # save the links to a csv file

In [ ]:
import csv 

links = []

with open('links_wo_duplicates.csv') as csvfile:    
	csvReader = csv.reader(csvfile)    
	for row in csvReader:        
		links.append(row[0])

links.pop(0)	#remove first element which is the header	   
print(links)

In [ ]:
print(links[:5])

In [86]:
dta = pd.DataFrame(columns=['Movie Name', 'Audience Score', 'Tomatometer Score', 'Rating', 'Genre',
                            'Original Language', 'Director', 'Producer', 'Writer', 'Release Date (Theaters)',
                           'Release Date (Streaming)', 'Box Office (Gross USA)', 'Runtime', 'Distributor',
                            'Sound Mix', 'Aspect Ratio', 'View the collection'])

Scrapeing of the movies start here

In [ ]:
from tqdm import tqdm

for link in tqdm(links[:5], desc='loading....'):
    driver.get(link)
    info_we = driver.find_elements(By.XPATH,'//score-board')
    movie_info = {'Movie Name':'','Audience Score':'','Tomatometer Score':''}
    movie_info['Movie Name'] = info_we[0].text.split('\n')[0]
    movie_info['Audience Score'] = info_we[0].get_attribute('audiencescore')
    movie_info['Tomatometer Score'] = info_we[0].get_attribute('tomatometerscore')
    webelement_list_val = []
    webelement_list_key = []
    webelement_list_key = driver.find_elements(By.XPATH,'//div[@data-qa="movie-info-item-label"]')
    webelement_list_val = driver.find_elements(By.XPATH,'//div[@data-qa="movie-info-item-value"]')
    key_list, val_list = [],[]
    for k,v in zip(webelement_list_key, webelement_list_val):
        key_list.append(k.text.strip(':'))
        val_list.append(v.text)
    info = dict(zip(key_list,val_list))

    total_info = {**movie_info,**info} 
    dta = dta.append(total_info, ignore_index=True)

In [ ]:
dta.head()

In [ ]:
dta.info()

Error if one column is empty, if empty use -1  (later it will be replaced NULL)

In [103]:
dta['Audience Score'].loc[dta['Audience Score'] == ""] = -1


In [ ]:
dta[['Tomatometer Score','Audience Score']]=dta[['Tomatometer Score','Audience Score']].astype(int)
print(dta[['Tomatometer Score','Audience Score']].dtypes)

In [ ]:
dta['Release Date (Streaming)'] = pd.to_datetime(dta['Release Date (Streaming)'])
print(dta['Release Date (Streaming)'].head())

In [106]:
def func(x):
    if type(x) != float:
        li = x.split(' ')
        li.remove(li[-1])
        time = ' '.join(li).replace(',','')
        return time
dta['Release Date (Theaters)'] = dta['Release Date (Theaters)'].apply(func)
dta['Release Date (Theaters)'] = pd.to_datetime(dta['Release Date (Theaters)'])

In [ ]:
dta['Rating'].loc[dta['Rating'].isnull()] = 'None'

In [108]:
dta['Rating'] = dta['Rating'].agg(lambda a : a.split(' ')[0])

In [ ]:
dta.Rating.value_counts()

In [ ]:
dta.head()

In [111]:
dta.to_csv('output/Setsfile.csv', index=False)